## Génération de données aléatoires pour renseigner la table personnes

In [1]:
from pyspark.sql import SparkSession
from faker import Faker
import pandas as pd
import random



### Demarrage de la session Spark. Si Spark pushdown active, verifier dans HUE qu'un Job Yarn est créé

In [2]:
# MASTER="yarn"           # Si Spark pushdown sur CML
MASTER="local[*]"       # Si execute parallele sur CML

spark = SparkSession.builder.appName("O_ingestion").master(MASTER).getOrCreate()


7.1.9 and 7.2.17 are the last CDP runtime releases where Spark 2 is supported.
Please migrate your Spark 2 applications to Spark 3.

Updating Spark 2 applications for Spark 3:
https://docs.cloudera.com/runtime/7.2.16/running-spark-applications/topics/spark-update-spark2-spark3.html



In [3]:
# Test de la connexion Spark
spark.sql("show databases").show()

Hive Session ID = ce5852d6-43eb-4779-aaf3-b0f5afcf4dd1


+------------------+
|      databaseName|
+------------------+
|           default|
|information_schema|
|           olivier|
|               sys|
+------------------+



### Paramètres d'initialisation de la génération de données

In [11]:
LIGNES=10000      # Nombre de lignes de la dataframe Spark de destination
PAS=1000          # Nombre de lignes de la dataframe Pandas intermédiaire

### Génération de données aléatoire à partir d'un Faker Python

In [12]:
%%time

fake = Faker()             # Initialisation du Faker

df = pd.DataFrame()        # Création de la dataframe Pandas pour les données du Faker

i = 0
while i < LIGNES:
          
    i = i + 1

    data = {
        'id': fake.random_number(digits=5),
        'age': random.randint(18, 70),
        'sexe': random.choice(['homme', 'femme']),
        'niveau_education': random.choice(['CAP/BEP', 'BACCALAUREAT', 'BTS/DUT',  'LICENSE', 'MAITRISE', 'MASTER', 'DOCTORAT']),
        'duree_chomage': random.randint(0, 24),
        'experience_ant': random.randint(0, 20),
        'competence': fake.job(),
        'formation': random.choice([True, False]),
        'taux_chomage_local': random.uniform(2, 20),
        'secteur_activite': fake.company_suffix(),
        'reseau_professionnel': random.randint(0, 100),
        'support_social': random.randint(0, 10),
        'retour_emploi': random.choice([True, False])
        }
    
    df = pd.concat( [df, pd.DataFrame(data, index=['id'])] )
    
    if (i%PAS == 0) : # A chaque remplissage de la dataframe pandas, chargement en bulk de la dataframe spark

       sdf = spark.createDataFrame(df) if (i == PAS) else sdf.unionAll(spark.createDataFrame(df))
         
       df  = df.drop(df.index)
    
       print ("Iteration: ", i/PAS)
        


    
    

Iteration:  1.0
Iteration:  2.0
Iteration:  3.0
Iteration:  4.0
Iteration:  5.0
Iteration:  6.0
Iteration:  7.0
Iteration:  8.0
Iteration:  9.0
Iteration:  10.0
CPU times: user 24.2 s, sys: 342 ms, total: 24.6 s
Wall time: 27.4 s


In [13]:
sdf.count()

10000

In [ ]:
TABLE="olivier.personnes"

sdf.write.format("parquet").mode("overwrite").saveAsTable(TABLE)


In [ ]:
sdf.describe()

In [ ]:
spark.stop()